In [1]:
# Thermostat UI

In [13]:
import requests
from ipywidgets import Button, HBox, IntProgress, IntSlider, Label, Layout, VBox

# Set the URL for the thermostat API
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"

# Get the current state of the thermostat from the API
thermostat = requests.get(THERMOSTAT_API_URL).json()

# Create labels to display the name and mode of the thermostat
name_label = Label(f"Name: {thermostat['name']}")
mode_label = Label(f"Mode: {thermostat['mode']}")

# Create a label and slider to display and adjust the desired temperature
desired_temp_label = Label(f"Desired Temperature: {thermostat['desired_temperature']}")
desired_temp_slider = IntSlider(
    value=thermostat["desired_temperature"], min=0, max=100, orientation="vertical"
)
desired_temp_box = VBox([HBox([desired_temp_slider]), desired_temp_label])

# Create a label and progress bar to display the actual temperature
actual_temp_label = Label(f"Actual Temperature: {thermostat['actual_temperature']}")
actual_temp_progress = IntProgress(
    value=thermostat["actual_temperature"],
    min=0,
    max=100,
    orientation="vertical",
    layout=Layout(height="200px"),
)
actual_temp_box = VBox([actual_temp_progress, actual_temp_label])

# Create a label and slider to display and adjust the desired humidity
desired_humidity_label = Label(f"Desired Humidity: {thermostat['desired_humidity']}")
desired_humidity_slider = IntSlider(
    value=thermostat["desired_humidity"], min=0, max=100, orientation="vertical"
)
desired_humidity_box = VBox([HBox([desired_humidity_slider]), desired_humidity_label])

# Create a label and progress bar to display the actual humidity
actual_humidity_label = Label(f"Actual Humidity: {thermostat['actual_humidity']}")
actual_humidity_progress = IntProgress(
    value=thermostat["actual_humidity"],
    min=0,
    max=100,
    orientation="vertical",
    layout=Layout(height="200px"),
)
actual_humidity_box = VBox([actual_humidity_progress, actual_humidity_label])

# Create a submit button to send changes to the API
submit_button = Button(description="Submit", button_style="danger")


def update_desired_temp(change):
    # Update the desired temperature in the thermostat state and update the label
    thermostat["desired_temperature"] = change.new
    desired_temp_label.value = f"Desired Temperature: {change.new}"


def update_desired_humidity(change):
    # Update the desired humidity in the thermostat state and update the label
    thermostat["desired_humidity"] = change.new
    desired_humidity_label.value = f"Desired Humidity: {change.new}"


def submit_changes(b):
    # Send changes in desired temperature and humidity to the API
    requests.put(
        THERMOSTAT_API_URL
        + "/desired_temperature"
        + f'?desired_temperature={thermostat["desired_temperature"]}'
    )

    requests.put(
        THERMOSTAT_API_URL
        + "/desired_humidity"
        + f'?desired_humidity={thermostat["desired_humidity"]}'
    )

    # Update the state of the thermostat from the API
    update_state()


def update_state():
    # Get the current state of the thermostat from the API and update all labels and progress bars
    thermostat.update(requests.get(THERMOSTAT_API_URL).json())

    name_label.value = f"Name: {thermostat['name']}"

    mode_label.value = f"Mode: {thermostat['mode']}"

    desired_temp_label.value = (
        f"Desired Temperature: {thermostat['desired_temperature']}"
    )

    actual_temp_label.value = f"Actual Temperature: {thermostat['actual_temperature']}"

    actual_temp_progress.value = thermostat["actual_temperature"]

    # Update humidity values
    desired_humidity_label.value = f"Desired Humidity: {thermostat['desired_humidity']}"

    actual_humidity_label.value = f"Actual Humidity: {thermostat['actual_humidity']}"

    actual_humidity_progress.value = thermostat["actual_humidity"]


# Set initial values for labels and progress bars
actual_temp_label.value = f"Actual Temperature: {thermostat['actual_temperature']}"
actual_temp_progress.value = thermostat["actual_temperature"]
# Observe changes in desired temperature slider
desired_temp_slider.observe(update_desired_temp, names="value")
# Observe changes in desired humidity slider
desired_humidity_slider.observe(update_desired_humidity, names="value")
# Handle clicks on the submit button
submit_button.on_click(submit_changes)

# Display all widgets
display(
    VBox(
        [
            name_label,
            mode_label,
            HBox(
                [
                    HBox([desired_temp_box, actual_temp_box]),
                    HBox([actual_humidity_box, desired_humidity_box]),
                    submit_button,
                ]
            ),
        ]
    )
)